In [8]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import snscrape.modules.twitter as sntwitter
from textblob import TextBlob
import warnings
warnings.filterwarnings("ignore")

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to C:\Users\Cheng
[nltk_data]     An\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

# SNSCRAPE to scrape through Twitter

In [ ]:
#Creating list to append tweet data
tweets_list = []
#keyword
keywords = ['SoyBeans','Rapeseed oil','Palm Oil','Sunflower oil', 'crude oil']
# No of tweets
noOfTweet = 50000


#Loop through important usernames:
user_names = open('Commodities.txt','r')


#for user in user_names:
def twitter_scrap (keyword,user):
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper("from:"+ user +" "+keyword ).get_items()):
        if i > int(noOfTweet):
            break
        tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.username, keyword])
usernames=[]        
for user in user_names:
    usernames.append(user)
    
for keyword in keywords:
    for user in usernames:
        print (keyword,user)
        twitter_scrap(keyword,user)

# Creating and cleaning the dataframe from the tweets list above 

In [ ]:
# Creating a dataframe from the tweets list above 
df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'Text', 'Username','oil'])
df['Datetime'] = pd.to_datetime(df['Datetime'],unit='ms').dt.tz_convert('Asia/Singapore')
df['Datetime'] = df['Datetime'].apply(lambda a: datetime.datetime.strftime(a,"%d-%m-%Y %H:%M:%S"))
df['Datetime'] = pd.to_datetime(df['Datetime'])          
df['Tweet Id'] = ('"'+ df['Tweet Id'].astype(str) + '"')      

# Create a function to clean the tweets
def cleanTxt(text):
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub('#', '', text) # Removing '#' hash tag
    text = re.sub('RT[\s]+', '', text) # Removing RT
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
    return text

df["Text"] = df["Text"].apply(cleanTxt)
df.head()

In [ ]:
# Saving it into csv
df.to_csv('tweet_commodities.csv')

# Sentiment Analysis

**NLTK**

In [ ]:
#Sentiment Analysis
def percentage(part,whole):
    return 100 * float(part)/float(whole)

#Iterating over the tweets in the dataframe

def apply_analysis(tweet):
    return SentimentIntensityAnalyzer().polarity_scores(tweet)

def sentimental_analysis(df):
    if df['neg'] > df['pos']:
        return 'Negative'
    elif df['pos'] > df['neg']:
        return 'Positive'
    elif df['pos'] == df['neg']:
        return 'Neutral'

In [ ]:
df[['neg','neu','pos','compound']] = df['summary'].apply(apply_analysis).apply(pd.Series)

df['Sentiment_NLTK'] = df.apply(sentimental_analysis, axis = 1)
df.head()

**Textblob**

In [157]:
def getSubjectivity(twt):
    return TextBlob(twt).sentiment.subjectivity
def getPolarity(twt):
    return TextBlob(twt).sentiment.polarity
def getSentiment(score):
    if score<0:
        return 'Negative'
    elif score==0:
        return 'Neutral'
    else:
        return 'Positive'

In [ ]:
df['Subjectivity']=df['summary'].apply(getSubjectivity)
df['Polarity']=df['summary'].apply(getPolarity)    
df['Sentiment_TB']=df['Polarity'].apply(getSentiment)

In [ ]:
# Saving Twitter CSV
df.to_csv('Twitter_sentiments.csv', encoding='utf-8-sig' ,index= False)

# Google API

In [1]:
import calendar
import numpy as np
import time
from GoogleNews import GoogleNews
import pandas as pd

In [3]:
def gns (m,y,last_day,key):
    googlenews = GoogleNews(start= "{}/01/{}".format(str(m).zfill(2),y),end="{}/{}/{}".format(str(m).zfill(2),last_day,y) )
    googlenews.search(key)
    data = googlenews.result()
    googlenews.clear()
    return(data)

In [ ]:
appended_data = []
month = list(np.arange(1,13))

# Set the timeline you want
year = list(np.arange(2018,2022))

news_update = -1

# Commodity [sunflower oil price, soybean price, palm oil price, coconut oil price, crude oil price]
# Change the key manually as google will block the api after awhile. Need to wait for sometime before searching another key.

key = 'palm oil price'
for y in year:
    for m in month:
        if not (y == 2022 and m > 3):
            last_day = calendar.monthrange(y, m)[1]
            print("{}/01/{}".format(str(m).zfill(2),y),"{}/{}/{}".format(str(m).zfill(2),last_day,y))
                   
            data = gns(m,y,last_day,key)
            print('1st check if there is search data from Google: '+ str(len(data)))
            while len(data) == 0:
                print('Retrying searching google for Data')
                time.sleep(10)
                data = gns(m,y,last_day,key)
                
            gn = pd.DataFrame(data)
            appended_data.append(gn)
            print('2nd check if there is search data from Google: ' + str(len(data)))
            time.sleep(10)
            
gn = pd.concat(appended_data)
gn = gn.reset_index(drop=True)
gn.to_csv('./Google News/PKO_.csv', index = False)

# Generating summary from the google news

In [143]:
import pandas as pd
import newspaper
from newspaper import Article
from newspaper import Config
import nltk
import random
nltk.download('punkt')
import re
import os

[nltk_data] Downloading package punkt to C:\Users\Cheng
[nltk_data]     An\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [144]:
# to circumvent from website blocking you
USER_AGENTS = '''Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36
Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2226.0 Safari/537.36
Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36
Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36
Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36
Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2214.93 Safari/537.36
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.124 Safari/537.36
Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0 Safari/537.36
Mozilla/5.0 (Windows NT 4.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0 Safari/537.36
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36
Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36
Mozilla/5.0 (X11; OpenBSD i386) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1944.0 Safari/537.36
Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.3319.102 Safari/537.36
Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.2309.372 Safari/537.36
Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.2117.157 Safari/537.36
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36
Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1866.237 Safari/537.36
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.137 Safari/4E423F
Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.116 Safari/537.36 Mozilla/5.0 (iPad; U; CPU OS 3_2 like Mac OS X; en-us) AppleWebKit/531.21.10 (KHTML, like Gecko) Version/4.0.4 Mobile/7B334b Safari/531.21.10
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.517 Safari/537.36
Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1664.3 Safari/537.36
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1664.3 Safari/537.36
Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1650.16 Safari/537.36
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1623.0 Safari/537.36
Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.17 Safari/537.36
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.62 Safari/537.36
Mozilla/5.0 (X11; CrOS i686 4319.74.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.57 Safari/537.36
Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.2 Safari/537.36
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1468.0 Safari/537.36
Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1467.0 Safari/537.36
Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1464.0 Safari/537.36
Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1500.55 Safari/537.36
Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36
Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36
Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_2) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.22 (KHTML, like Gecko) Chrome/19.0.1047.0 Safari/535.22
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.21 (KHTML, like Gecko) Chrome/19.0.1042.0 Safari/535.21
Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.21 (KHTML, like Gecko) Chrome/19.0.1041.0 Safari/535.21
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/535.2 (KHTML, like Gecko) Chrome/18.6.872.0 Safari/535.2 UNTRUSTED/1.0 3gpp-gba UNTRUSTED/1.0
Mozilla/5.0 (Macintosh; AMD Mac OS X 10_8_2) AppleWebKit/535.22 (KHTML, like Gecko) Chrome/18.6.872
Mozilla/5.0 (X11; CrOS i686 1660.57.0) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.46 Safari/535.19
Mozilla/5.0 (Windows NT 6.0; WOW64) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.45 Safari/535.19
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_2) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.45 Safari/535.19
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.45 Safari/535.19
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.166 Safari/535.19
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_5_8) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.151 Safari/535.19
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.19 (KHTML, like Gecko) Ubuntu/11.10 Chromium/18.0.1025.142 Chrome/18.0.1025.142 Safari/535.19
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.11 Safari/535.19
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.66 Safari/535.11
Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.66 Safari/535.11
Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.66 Safari/535.11
Mozilla/5.0 (Windows NT 6.2) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.66 Safari/535.11
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.66 Safari/535.11
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.66 Safari/535.11
Mozilla/5.0 (Windows NT 6.0; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.66 Safari/535.11
Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.66 Safari/535.11
Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.66 Safari/535.11
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.66 Safari/535.11
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_2) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.66 Safari/535.11
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.66 Safari/535.11
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_5_8) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.66 Safari/535.11
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.11 (KHTML, like Gecko) Ubuntu/11.10 Chromium/17.0.963.65 Chrome/17.0.963.65 Safari/535.11
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.11 (KHTML, like Gecko) Ubuntu/11.04 Chromium/17.0.963.65 Chrome/17.0.963.65 Safari/535.11
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.11 (KHTML, like Gecko) Ubuntu/10.10 Chromium/17.0.963.65 Chrome/17.0.963.65 Safari/535.11
Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.11 (KHTML, like Gecko) Ubuntu/11.10 Chromium/17.0.963.65 Chrome/17.0.963.65 Safari/535.11
Mozilla/5.0 (Windows NT 5.1) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.700.3 Safari/534.24
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.699.0 Safari/534.24
Mozilla/5.0 (Windows NT 6.0; WOW64) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.699.0 Safari/534.24
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_6) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.698.0 Safari/534.24
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.697.0 Safari/534.24
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.696.71 Safari/534.24
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.696.68 Safari/534.24
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_7) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.696.68 Safari/534.24
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_5_8) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.696.68 Safari/534.24
Mozilla/5.0 Slackware/13.37 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/11.0.696.50
Mozilla/5.0 (Windows NT 5.1) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.696.43 Safari/534.24
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.696.34 Safari/534.24
Mozilla/5.0 (Windows NT 6.0; WOW64) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.696.34 Safari/534.24
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.696.3 Safari/534.24
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.696.3 Safari/534.24
Mozilla/5.0 (Windows NT 6.0) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.696.3 Safari/534.24
Mozilla/5.0 (X11; Linux i686) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.696.14 Safari/534.24
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.696.12 Safari/534.24
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_6) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.696.12 Safari/534.24
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/534.24 (KHTML, like Gecko) Ubuntu/10.04 Chromium/11.0.696.0 Chrome/11.0.696.0 Safari/534.24
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.696.0 Safari/534.24
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.694.0 Safari/534.24
Mozilla/5.0 (X11; Linux i686) AppleWebKit/534.23 (KHTML, like Gecko) Chrome/11.0.686.3 Safari/534.23
Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.21 (KHTML, like Gecko) Chrome/11.0.682.0 Safari/534.21
Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.21 (KHTML, like Gecko) Chrome/11.0.678.0 Safari/534.21
Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_7_0; en-US) AppleWebKit/534.21 (KHTML, like Gecko) Chrome/11.0.678.0 Safari/534.21
Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/534.20 (KHTML, like Gecko) Chrome/11.0.672.2 Safari/534.20
Mozilla/5.0 (Windows NT) AppleWebKit/534.20 (KHTML, like Gecko) Chrome/11.0.672.2 Safari/534.20
Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_6; en-US) AppleWebKit/534.20 (KHTML, like Gecko) Chrome/11.0.672.2 Safari/534.20
Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.20 (KHTML, like Gecko) Chrome/11.0.669.0 Safari/534.20
Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.19 (KHTML, like Gecko) Chrome/11.0.661.0 Safari/534.19
Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.18 (KHTML, like Gecko) Chrome/11.0.661.0 Safari/534.18
Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_6; en-US) AppleWebKit/534.18 (KHTML, like Gecko) Chrome/11.0.660.0 Safari/534.18
Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.17 (KHTML, like Gecko) Chrome/11.0.655.0 Safari/534.17
Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_4; en-US) AppleWebKit/534.17 (KHTML, like Gecko) Chrome/11.0.655.0 Safari/534.17
Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.17 (KHTML, like Gecko) Chrome/11.0.654.0 Safari/534.17
Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/534.17 (KHTML, like Gecko) Chrome/11.0.652.0 Safari/534.17
Mozilla/4.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/11.0.1245.0 Safari/537.36
Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.17 (KHTML, like Gecko) Chrome/10.0.649.0 Safari/534.17
Mozilla/5.0 (Windows; U; Windows NT 6.1; de-DE) AppleWebKit/534.17 (KHTML, like Gecko) Chrome/10.0.649.0 Safari/534.17
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.82 Safari/534.16
Mozilla/5.0 (X11; U; Linux armv7l; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.204 Safari/534.16
Mozilla/5.0 (X11; U; FreeBSD x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.204 Safari/534.16
Mozilla/5.0 (X11; U; FreeBSD i386; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.204 Safari/534.16
Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_5; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.204
Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.134 Safari/534.16
Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.134 Safari/534.16
Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.134 Safari/534.16
Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_6; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.134 Safari/534.16
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Ubuntu/10.10 Chromium/10.0.648.133 Chrome/10.0.648.133 Safari/534.16
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.133 Safari/534.16
Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Ubuntu/10.10 Chromium/10.0.648.133 Chrome/10.0.648.133 Safari/534.16
Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.133 Safari/534.16
Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.133 Safari/534.16
Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_3; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.133 Safari/534.16
Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_2; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.133 Safari/534.16
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Ubuntu/10.10 Chromium/10.0.648.127 Chrome/10.0.648.127 Safari/534.16
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.127 Safari/534.16
Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_4; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.127 Safari/534.16
Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_5_8; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.127 Safari/534.16
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.11 Safari/534.16
Mozilla/5.0 (Windows; U; Windows NT 6.1; ru-RU; AppleWebKit/534.16; KHTML; like Gecko; Chrome/10.0.648.11;Safari/534.16)
Mozilla/5.0 (Windows; U; Windows NT 6.1; ru-RU) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.11 Safari/534.16
Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.11 Safari/534.16
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Ubuntu/10.10 Chromium/10.0.648.0 Chrome/10.0.648.0 Safari/534.16
Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Ubuntu/10.10 Chromium/10.0.648.0 Chrome/10.0.648.0 Safari/534.16
Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_4; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.0 Safari/534.16
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Ubuntu/10.10 Chromium/10.0.642.0 Chrome/10.0.642.0 Safari/534.16
Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_5; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.639.0 Safari/534.16
Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.638.0 Safari/534.16
Mozilla/5.0 (X11; U; Linux i686 (x86_64); en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.634.0 Safari/534.16
Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.634.0 Safari/534.16
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 SUSE/10.0.626.0 (KHTML, like Gecko) Chrome/10.0.626.0 Safari/534.16
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.15 (KHTML, like Gecko) Chrome/10.0.613.0 Safari/534.15
Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/534.15 (KHTML, like Gecko) Ubuntu/10.10 Chromium/10.0.613.0 Chrome/10.0.613.0 Safari/534.15
Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/534.15 (KHTML, like Gecko) Ubuntu/10.04 Chromium/10.0.612.3 Chrome/10.0.612.3 Safari/534.15
Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/534.15 (KHTML, like Gecko) Chrome/10.0.612.1 Safari/534.15
Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/534.15 (KHTML, like Gecko) Ubuntu/10.10 Chromium/10.0.611.0 Chrome/10.0.611.0 Safari/534.15
Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/10.0.602.0 Safari/534.14
Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/10.0.601.0 Safari/534.14
Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/10.0.601.0 Safari/534.14
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/540.0 (KHTML,like Gecko) Chrome/9.1.0.0 Safari/540.0
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/540.0 (KHTML, like Gecko) Ubuntu/10.10 Chrome/9.1.0.0 Safari/540.0
Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/9.0.601.0 Safari/534.14
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Ubuntu/10.10 Chromium/9.0.600.0 Chrome/9.0.600.0 Safari/534.14
Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/9.0.600.0 Safari/534.14
Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.599.0 Safari/534.13
Mozilla/5.0 (Windows; U; Windows NT 5.1; en-CA) AppleWebKit/534.13 (KHTML like Gecko) Chrome/9.0.597.98 Safari/534.13
Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.84 Safari/534.13
Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.44 Safari/534.13
Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.19 Safari/534.13
Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.15 Safari/534.13
Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_5; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.15 Safari/534.13
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.107 Safari/534.13 v1416758524.9051
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.107 Safari/534.13 v1416748405.3871
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.107 Safari/534.13 v1416670950.695
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.107 Safari/534.13 v1416664997.4379
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.107 Safari/534.13 v1333515017.9196
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.0 Safari/534.13
Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US)  AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.0 Safari/534.13
Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.0 Safari/534.13
Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.0 Safari/534.13
Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_5; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.0 Safari/534.13
Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_4; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.0 Safari/534.13
Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.596.0 Safari/534.13
Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Ubuntu/10.04 Chromium/9.0.595.0 Chrome/9.0.595.0 Safari/534.13
Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Ubuntu/9.10 Chromium/9.0.592.0 Chrome/9.0.592.0 Safari/534.13
Mozilla/5.0 (X11; U; Windows NT 6; en-US) AppleWebKit/534.12 (KHTML, like Gecko) Chrome/9.0.587.0 Safari/534.12
Mozilla/5.0 (Windows  U  Windows NT 5.1  en-US) AppleWebKit/534.12 (KHTML, like Gecko) Chrome/9.0.583.0 Safari/534.12'''.split('\n')

In [ ]:
directory = './Google News'
appended_news= []
for filename in os.listdir(directory):
    f = directory+'/'+filename
    print(f)
    df = pd.read_csv(f)
    USER_AGENT = random.choice(USER_AGENTS)
    config = Config()
    config.browser_user_agent = USER_AGENT
    config.request_timeout = 15
    print(filename[:3])
    for url in df['link']:
        #print(url)
        base_url = re.search("^.+?[^\/:](?=[?\/]|$)",url)[0]
        print(base_url)
        article_urls = set()
        euronews = newspaper.build(base_url, config=config, memoize_articles=False, language='en')
        article = Article(url,config = config , memoize_articles=False, language='en')
        try:
            article.download()
            article.parse()
            article.nlp()
            df.loc[df['link'] == url,"summary"] = article.summary
        except:
            df.loc[df['link'] == url,"summary"] = ''
    df['type'] = filename[:3] 
    
    appended_news.append(df)

news_sum = pd.concat(appended_news)
        

In [ ]:
# Saving it into csv
news_sum.to_csv('Google_news_summary.csv',index = False)

# Sentiment Analysis for Google News

In [ ]:
df = pd.read_csv('./Google_news_summary.csv')

# Drop Null summary.
df = df.loc[df['summary'].notnull()]

#Sentiment NLTK
df['Sentiment_NLTK'] = df.apply(sentimental_analysis, axis = 1)

#Sentiment Textblob
df['Subjectivity']=df['summary'].apply(getSubjectivity)
df['Polarity']=df['summary'].apply(getPolarity)    
df['Sentiment_TB']=df['Polarity'].apply(getSentiment)

# Saving it to csv
df.to_csv('Google_news_sentiment.csv', encoding='utf-8-sig' ,index= False)

# Sorting Barchart News

**IF**

In [146]:
df = pd.read_csv('./IF_combined.csv')

# Drop null data
df = df.dropna(subset=['preview'])
df = df.loc[~df['preview'].str.contains(r'\b(Actionable|Action|Actiona)\b')]
df.head()

,Unnamed: 0,newsid,timestamp,source,categories,subcategories,headline,isexternal,organization,canonicalurl,relatedsymbols,preview,headlineurl,pdfurl
0,0,468755,2020-10-20T14:04:30-05:00,IF,['futures'],"['energies', 'grains', 'softs']",Where Weather Matters Most (what these 2 marke...,False,BestWeatherInc.com,https://www.barchart.com/story/news/468755/whe...,[],"October 20, 2020 Wheat and Sugar Prices are...",NaN,NaN
1,1,468673,2020-10-20T13:55:53-05:00,IF,['futures'],['energies'],Looking To Buy Heating Oil,False,Seery Futures,https://www.barchart.com/story/news/468673/loo...,[],Risk in your favor soon...,NaN,NaN
3,3,466642,2020-10-21T14:05:03-05:00,IF,['futures'],['energies'],Do What Is Necessary. The Energy Report 10...,False,The PRICE Futures Group,https://www.barchart.com/story/news/466642/do-...,[],PRICE Futures Group · Do What Is Necessary. Th...,https://blog.pricegroup.com/2020/10/20/do-what...,NaN
4,4,463294,2020-10-20T05:44:22-05:00,IF,['futures'],"['energies', 'financials', 'indices']",No Stimulus in Sight,False,Market Tea Leaves,https://www.barchart.com/story/news/463294/no-...,[],Yesterday the markets fell with no economic st...,NaN,NaN
5,5,457324,2020-10-19T09:45:32-05:00,IF,['futures'],['energies'],Natural gas Weekly Forecast,False,Kolhanov.com,https://www.barchart.com/story/news/457324/nat...,[],"An uptrend will start as soon, as the market r...",NaN,NaN


In [147]:
# Calculating the occurance of the type of grains

df['Gas'] = (df['preview']+df['headline']+df['subcategories']).str.lower().str.count('gas')
df['Oil'] = (df['preview']+df['headline']+df['subcategories']).str.lower().str.count('oil|crude|clz|clf|hoz|hof')
df['Grains'] = (df['preview']+df['headline']+df['subcategories']).str.lower().str.count('grain|crop|bean|coffee|weather')
df['Ethanol'] = (df['preview']+df['headline']+df['subcategories']).str.lower().str.count('ethanol')
df['Gold'] = (df['preview']+df['headline']+df['subcategories']).str.lower().str.count('gold')
df['Silver'] = (df['preview']+df['headline']+df['subcategories']).str.lower().str.count('silver')
df['Platinum'] = (df['preview']+df['headline']+df['subcategories']).str.lower().str.count('platinum')
df['Corn'] = (df['preview']+df['headline']+df['subcategories']).str.lower().str.count('corn')
df['Wheat'] = (df['preview']+df['headline']+df['subcategories']).str.lower().str.count('wheat')
df['Soy'] = (df['preview']+df['headline']+df['subcategories']).str.lower().str.count('soy')
df['S&P'] = (df['preview']+df['headline']+df['subcategories']).str.lower().str.count('s&p|dow')
df['Energy'] = (df['preview']+df['headline']+df['subcategories']).str.lower().str.count('energ')
df['Market'] = (df['preview']+df['headline']+df['subcategories']).str.lower().str.count('market')
df['Commodity'] = (df['preview']+df['headline']+df['subcategories']).str.lower().str.count('commodi')
df['Cattle'] = (df['preview']+df['headline']+df['subcategories']).str.lower().str.count('cattle|beef')

In [148]:
s = df[['Gas', 'Oil', 'Grains', 'Ethanol', 'Gold', 'Silver', 'Platinum', 'Corn', 'Wheat','Soy','S&P', 'Energy','Market','Cattle','Commodity']].idxmax(axis=1)

df['Type'] = s
df.head()


,Unnamed: 0,newsid,timestamp,source,categories,subcategories,headline,isexternal,organization,canonicalurl,...,Platinum,Corn,Wheat,Soy,S&P,Energy,Market,Commodity,Cattle,Type
0,0,468755,2020-10-20T14:04:30-05:00,IF,['futures'],"['energies', 'grains', 'softs']",Where Weather Matters Most (what these 2 marke...,False,BestWeatherInc.com,https://www.barchart.com/story/news/468755/whe...,...,0,0,1,0,0,1,1,1,0,Grains
1,1,468673,2020-10-20T13:55:53-05:00,IF,['futures'],['energies'],Looking To Buy Heating Oil,False,Seery Futures,https://www.barchart.com/story/news/468673/loo...,...,0,0,0,0,0,1,0,0,0,Oil
3,3,466642,2020-10-21T14:05:03-05:00,IF,['futures'],['energies'],Do What Is Necessary. The Energy Report 10...,False,The PRICE Futures Group,https://www.barchart.com/story/news/466642/do-...,...,0,0,0,0,0,3,0,0,0,Energy
4,4,463294,2020-10-20T05:44:22-05:00,IF,['futures'],"['energies', 'financials', 'indices']",No Stimulus in Sight,False,Market Tea Leaves,https://www.barchart.com/story/news/463294/no-...,...,0,0,0,0,0,1,1,0,0,Energy
5,5,457324,2020-10-19T09:45:32-05:00,IF,['futures'],['energies'],Natural gas Weekly Forecast,False,Kolhanov.com,https://www.barchart.com/story/news/457324/nat...,...,0,0,0,0,0,1,1,0,0,Gas


In [149]:
# Saving as csv file
df.to_csv('IFsorted.csv',index=False)

**Brug**

In [150]:
df = pd.read_csv('./BRUG_combined.csv')

# Drop null data
df = df.dropna(subset=['preview'])

In [151]:
# Calculating the occurance of the type of grains

df['Wheat'] = (df['preview']+df['headline']).str.lower().str.count('wheat')
df['Soy'] = (df['preview']+df['headline']).str.lower().str.count('soy|bean')
df['Corn'] = (df['preview']+df['headline']).str.lower().str.count('corn')

In [152]:
# data of intereset
s = df[['Wheat', 'Soy', 'Corn']].idxmax(axis=1)

df['Type'] = s

In [153]:
df.to_csv('BRUGsorted.csv',index=False)

# Sentiment Analysis for Barchart (IF & BRUG)

In [154]:
if_news = pd.read_csv('IFsorted.csv')
brug_news = pd.read_csv('BRUGsorted.csv')

text_list = ['headline','preview']

**NLTK Slight Modification**

In [155]:
#Sentiment Analysis
def percentage(part,whole):
    return 100 * float(part)/float(whole)

#Iterating over the text in the dataframe

def apply_analysis(text):
    return SentimentIntensityAnalyzer().polarity_scores(text)

text_list = ['headline','preview']
def sentimental_analysis(df, text):
    if df['{0} neg'.format(text)] > df['{0} pos'.format(text)]:
        return 'Negative'
    elif df['{0} pos'.format(text)] > df['{0} neg'.format(text)]:
        return 'Positive'
    elif df['{0} pos'.format(text)] == df['{0} neg'.format(text)]:
        return 'Neutral'

**IF News**

In [158]:
#NLTK
for x in text_list:
    if_news[['{0} neg'.format(x),'{0} neu'.format(x),'{0} pos'.format(x),'{0} compound'.format(x)]] = if_news[x].apply(apply_analysis).apply(pd.Series)
    if_news['{0} Sentiment_NLTK'.format(x)] = if_news.apply(sentimental_analysis,text=x, axis = 1)

#TextBlob    
for x in text_list:    
    if_news['{0} Subjectivity'.format(x)]=if_news[x].apply(getSubjectivity)
    if_news['{0} Polarity'.format(x)]=if_news[x].apply(getPolarity)    
    if_news['{0} Sentiment_TB'.format(x)]=if_news['{0} Polarity'.format(x)].apply(getSentiment)
    
if_news.to_csv('./IF_sentiment.csv', index = False)

**Brug News**

In [ ]:
#NLTK
for x in text_list:
    brug_news[['{0} neg'.format(x),'{0} neu'.format(x),'{0} pos'.format(x),'{0} compound'.format(x)]] = brug_news[x].apply(apply_analysis).apply(pd.Series)
    brug_news['{0} Sentiment_NLTK'.format(x)] = brug_news.apply(sentimental_analysis,text=x, axis = 1)

#TextBlob    
for x in text_list:    
    brug_news['{0} Subjectivity'.format(x)]=brug_news[x].apply(getSubjectivity)
    brug_news['{0} Polarity'.format(x)]=brug_news[x].apply(getPolarity)    
    brug_news['{0} Sentiment_TB'.format(x)]=brug_news['{0} Polarity'.format(x)].apply(getSentiment)
    
brug_news.to_csv('./brug_sentiment.csv' , index = False)

# Averaging Sentiments

In [122]:
twitter = pd.read_csv('./Twitter_sentiment.csv')
gn = pd.read_csv('./Google_news_sentiment.csv')
brug = pd.read_csv('./brug_sentiment.csv')
if_ = pd.read_csv('./IF_sentiment.csv')

In [123]:
if_['date'] = pd.to_datetime(if_['timestamp'], utc=True).dt.date
twitter['date'] = pd.to_datetime(twitter['Datetime'], utc=True).dt.date
brug['date'] = pd.to_datetime(brug['timestamp'], utc=True).dt.date
gn['date'] = pd.to_datetime(gn['date'],format = '%d/%m/%Y').dt.date

twitter = twitter.loc[pd.to_datetime(twitter['date']).between('2017-12-31','2022-04-01', inclusive=False)]

In [124]:
def averaging_sentiments (df,date_,type_,nltk_sentiment):
    cleanup_nums = {nltk_sentiment:    {"Negative": -1, "Neutral": 0,"Positive": 1}}
    df = df.replace(cleanup_nums)
    df[type_] = df[type_].str.lower()
    df = df.groupby([date_,type_]).agg({nltk_sentiment: np.average})
    df.columns = ['Sentiment_NLTK']
    df = df.reset_index(level=[0,1])
    return df

In [125]:

twitter_ = averaging_sentiments(twitter,'date','Type','Sentiment_NLTK')
if__ = averaging_sentiments(if_,'date','Type','preview Sentiment_NLTK')
brug_ = averaging_sentiments(brug,'date','Type','preview Sentiment_NLTK')

#Cleanup for google news
cleanup_type = {'Type':{'CLZ': 'crude oil','CNO':'coconut oil','PO_':'palm oil', 'RSO':'rapeseed oil','SBO':'soy','SFO':'sunflower oil'}}

gn = gn.replace(cleanup_type)
gn_ =  averaging_sentiments(gn,'date','Type','Sentiment_NLTK')
gn_

,date,Type,Sentiment_NLTK
0,2018-01-01,coconut oil,0.500000
1,2018-01-01,crude oil,-1.000000
2,2018-01-01,palm oil,0.000000
3,2018-01-01,rapeseed oil,0.333333
4,2018-01-01,soy,0.333333
...,...,...,...
1735,2022-03-16,crude oil,-1.000000
1736,2022-03-16,soy,1.000000
1737,2022-03-16,sunflower oil,-1.000000
1738,2022-03-17,coconut oil,1.000000


In [126]:
# Merging all the DF together
sentiments = pd.merge(if__,twitter_,how="outer",on = ['date','Type'])
sentiments = pd.merge(sentiments,brug_,how="outer",on = ['date','Type'])
sentiments = pd.merge(sentiments,gn_,how="outer",on = ['date','Type'])

In [127]:
sentiments.sort_values(by = 'date')
sentiments['avg'] = sentiments.iloc[:,2:6].mean(axis=1)
sentiments['count'] = sentiments.iloc[:,2:6].count(axis=1)
sentiments = sentiments[['date', 'Type', 'avg', 'count']]
sentiments.to_csv('sentiments_.csv',index= False)

In [128]:
sentiments.columns

Index(['date', 'Type', 'avg', 'count'], dtype='object')

In [129]:
from datetime import datetime , timedelta

In [130]:
sent_ = pd.read_csv('./sentiments_.csv')

In [131]:
sent_['date'] = pd.to_datetime(sent_['date'])

In [132]:
date_time_str = '01/01/2020'

date_time_obj = datetime.strptime(date_time_str, '%m/%d/%Y')

# Set the start date and the end date

start = '01/01/2018'
end = '03/19/2022'

start = datetime.strptime(start, '%m/%d/%Y')
end = datetime.strptime(end, '%m/%d/%Y')
days_left = end - start
days_left.days

1538

In [133]:
dates = []
date_time_str = '01/01/2018'
date_time_obj = datetime.strptime(date_time_str, '%m/%d/%Y')
for x in range (days_left.days):
    dates.append(date_time_obj )
    date_time_obj = date_time_obj + timedelta(days = 1)

In [134]:
sent_date = pd.DataFrame(dates,columns=['date'])
sent_date['date']

0      2018-01-01
1      2018-01-02
2      2018-01-03
3      2018-01-04
4      2018-01-05
          ...    
1533   2022-03-14
1534   2022-03-15
1535   2022-03-16
1536   2022-03-17
1537   2022-03-18
Name: date, Length: 1538, dtype: datetime64[ns]

In [135]:
sent_.columns

Index(['date', 'Type', 'avg', 'count'], dtype='object')

In [136]:
sent_.sort_values(by = 'date', ascending=True)

for x in sent_date['date'].unique():
    temp  = sent_.loc[sent_['date'] == x]
    for y in temp['Type'].unique():
        #new_df_.loc[new_df['date'] == x, y] = temp['type']
        sent_.loc[sent_['date'] == x, y] = ((((temp.loc[temp['Type'] == y])['avg']).values)[0])
        sent_.loc[sent_['date'] == x, '{}_Count'.format(y)] = ((((temp.loc[temp['Type'] == y])['count']).values)[0])

In [137]:
# place 0 for days without news/ twitter
sent_ = sent_.drop(['Type','avg','count'], axis=1)
sent_ = sent_.fillna(0)
sent_.to_csv('Sentiments_new_.csv',index=False)

In [139]:
# if you want it to be binary values if not dont run it

"""def sentimental_analysis(df,x):
    df.loc[df[x] < 0 , '{}'.format(x)] = -1
    df.loc[df[x] == 0 , '{}'.format(x)] = 0
    df.loc[df[x] > 0 , '{}'.format(x)] = 1
    return df

for x in ['crude oil', 'palm oil', 'soy', 'sunflower oil', 'coconut oil',
       'rapeseed oil', 'energy', 'corn', 'wheat', 'gas', 'grains', 'ethanol',
       'market', 'gold', 's&p', 'commodity', 'cattle', 'silver', 'platinum']:
    sent_[x] = sent_[x].astype(float)
    test = sentimental_analysis(sent_,x)"""

"def sentimental_analysis(df,x):\n    df.loc[df[x] < 0 , '{}'.format(x)] = -1\n    df.loc[df[x] == 0 , '{}'.format(x)] = 0\n    df.loc[df[x] > 0 , '{}'.format(x)] = 1\n    return df\n\nfor x in ['crude oil', 'palm oil', 'soy', 'sunflower oil', 'coconut oil',\n       'rapeseed oil', 'energy', 'corn', 'wheat', 'gas', 'grains', 'ethanol',\n       'market', 'gold', 's&p', 'commodity', 'cattle', 'silver', 'platinum']:\n    sent_[x] = sent_[x].astype(float)\n    test = sentimental_analysis(sent_,x)"

In [ ]:
aggs = {}
for x in sent_.columns:
    if "Count" not in x and x != 'date':
        aggs[x] = np.mean
    if "Count" in x:  
        aggs[x] = np.sum

sentiment_subset = sent_.groupby([pd.Grouper(key='date', freq='M')]).agg(aggs).reset_index()


In [141]:
# Monthly sentiment dataset
sentiment_subset.to_csv('Sentiments_new_m_continuous.csv',index = False)